In [18]:
import spacy
import en_core_web_sm
nlp = en_core_web_sm.load(disable=[ "ner","tagger",'textcat'])
from spellchecker import SpellChecker
import nltk
from nltk.tokenize import word_tokenize
import re
import string
import unicodedata
import emoji as emoji
import regex
import spacy
from spacy.tokens import Token
from bs4 import BeautifulSoup
import pandas as pd
from nltk.corpus import stopwords
from tqdm import tqdm
tqdm.pandas()
from sklearn.model_selection import train_test_split
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from pandas import Panel
from sklearn.svm import LinearSVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import KFold
import time
import numpy as np
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
import warnings
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
warnings.filterwarnings("ignore")

In [21]:

#THE HYPERPARAMETER BASE_ESTIMATOR CAN BE CHANGED BUT FOR SIMPLICITY
def ada_grid_search(hyper_comb,X_train,y_train,file_name):
    start=time.time()
    all_res_cv = []
    cv = KFold(n_splits=5, shuffle=True, random_state=1848773) # 5 fold
    grid = ParameterGrid(hyper_comb) #all possible combination of parameters chosen
    for hypers in tqdm(grid):
        learning_rate=hypers['learning_rate']
        n_estimators=hypers['n_estimators']
        val_f1_list = []
        train_f1_list = []
        val_recall_list=[]
        train_recall_list=[]
        val_accuracy_list=[]
        train_accuracy_list=[]
        val_pre_list=[]
        train_pre_list=[]
        

        for train_index, test_index in (cv.split(X_train)): # cross validation
            
            #take 80% and 20%
            X_train_cv, X_val, y_train_cv, y_val = X_train[train_index], X_train[test_index], y_train[train_index], y_train[test_index]
            #fit the model with the chosen parameters
            model=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=learning_rate,n_estimators=n_estimators, random_state=1848773).fit(X_train_cv,y_train_cv)

            # make prediction and compute F1
            pred_train = model.predict(X_train_cv)
            pred_val = model.predict(X_val)
            #F1 SCORE
            f1_train = f1_score(y_train_cv, pred_train, average='micro')
            train_f1_list.append(f1_train)
            f1_val = f1_score(y_val, pred_val, average='micro')
            val_f1_list.append(f1_val)
            #RECALL VALUES
            recall_train=recall_score(y_train_cv,pred_train, average='micro')
            train_recall_list.append(recall_train)
            recall_val=recall_score(y_val,pred_val, average='micro')
            val_recall_list.append(recall_val)
            #ACCURACY
            accuracy_train=accuracy_score(y_train_cv,pred_train)
            train_accuracy_list.append(accuracy_train)
            accuracy_val=accuracy_score(y_val,pred_val)
            val_accuracy_list.append(accuracy_val)
            #PRECISION
            pre_train=precision_score(y_train_cv,pred_train, average='micro')
            train_pre_list.append(pre_train)
            pre_val=precision_score(y_val,pred_val, average='micro')
            val_pre_list.append(pre_val)



            all_res_cv.append({'learning_rate':learning_rate,
                                'n_estimators':n_estimators,
                               'val_f1':np.mean(val_f1_list), 
                               'train_f1':np.mean(train_f1_list),
                               'val_recall':np.mean(val_recall_list), 
                               'train_recall':np.mean(train_recall_list),
                               'val_acc':np.mean(val_accuracy_list), 
                               'train_acc':np.mean(train_accuracy_list),
                               'val_pre':np.mean(val_pre_list),
                              'train_pre':np.mean(train_pre_list)})
            '''
            print(({'learning_rate':learning_rate,
                                'n_estimators':n_estimators,
                'val_f1':np.mean(val_f1_list), 'train_f1':np.mean(train_f1_list)}))
            '''


        # after the 5th iteration we make the mean of the collected errors to find a more reliable value for the error
    end=time.time()-start
    df = pd.DataFrame(all_res_cv) # collect all results in a dataframe
    df.to_csv(file_name, index=False) # save dataframe in csv with results

    return print('File created grid_search_cv')

##combination of hyperparameters


In [12]:
train=pd.read_csv('pre_train.csv')

In [13]:
sub=list(train['prep_text'])
sub[5115]='Err:509'
train['prep_text']=sub

## bow_models

In [14]:
X = train['prep_text']
y = train['target']

In [15]:
count_vectorizer = feature_extraction.text.CountVectorizer()
X_train_=count_vectorizer.fit_transform(X)

In [16]:
hyper_comb={ 'learning_rate':list(np.arange(0.6,3,0.2)),
            'n_estimators':list(np.arange(30,70,1))}

In [22]:
ada_grid_search(hyper_comb,X_train_,y,'ada_grid_search_cv_bow.csv')

100%|██████████| 480/480 [2:57:29<00:00, 22.19s/it]  


File created grid_search_cv


In [25]:
max(list(pd.read_csv('ada_grid_search_cv_bow.csv')['val_f1']))

0.7606752527819169

### tf_idf

In [106]:
tfidf_transformer = TfidfTransformer()
train_tfidf_ = tfidf_transformer.fit_transform(X_train_)

In [ ]:
ada_grid_search(hyper_comb,train_tfidf_,y,'ada_grid_search_cv_tfidf.csv')

In [24]:
(list(pd.read_csv('ada_grid_search_cv_tfidf.csv')['val_f1']))

FileNotFoundError: [Errno 2] File b'ada_grid_search_cv_tfidf.csv' does not exist: b'ada_grid_search_cv_tfidf.csv'